###ML1_1: 
https://www.hackerrank.com/challenges/capturing-non-capturing-groups/problem?isFullScreen=true

In [7]:
import re

# Ввод строки
test_string = input("Введите строку:")

# Регулярное выражение для поиска двух или более "ok" подряд
pattern = r'(ok){2,}'

# Проверка на совпадение
if re.search(pattern, test_string):
    print("Совпадение найдено!")
else:
    print("Совпадение не найдено.")


Введите строку: okok


Совпадение найдено!


###ML1_2: 
https://www.hackerrank.com/challenges/branch-reset-groups/problem?isFullScreen=true

In [10]:
import re

# Ввод строки
test_string = input("Введите строку: ")

# Регулярное выражение для проверки строки
pattern = r'^\d{2}([-:.]|---)\d{2}\1\d{2}\1\d{2}$'

# Проверка на совпадение
if re.match(pattern, test_string):
    print("Совпадение найдено!")
else:
    print("Совпадение не найдено.")


Введите строку:  12.34.56.78


Совпадение найдено!


###ML1_3: 
https://www.hackerrank.com/challenges/detect-html-links/problem?isFullScreen=true

In [16]:
from bs4 import BeautifulSoup
import sys

# Чтение количества строк HTML-фрагмента
n = int(input())

# Чтение всех строк HTML-фрагмента
html_content = ''.join([input().strip() for _ in range(n)])

# Создание объекта BeautifulSoup для разбора HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Проход по всем тегам <a>
for link in soup.find_all('a', href=True):
    # Извлечение ссылки
    href = link['href'].strip()
    # Извлечение текста внутри тега, очищенного от вложенных тегов
    text = link.get_text().strip()
    # Печать результата
    print(f"{href},{text}")


  2
 <p><a href="https://www.hackerrank.com">Example Link</a></p>
 <div><a href="https://www.hackerrank.com/another">Another Link</a></div>


https://www.hackerrank.com,Example Link
https://www.hackerrank.com/another,Another Link


###ML1_4: 
Реализовать stemming, lemmatization & BoW на следующем датасете: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

In [11]:
# Импорт необходимых библиотек
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import string
import os

# 1. Установка и загрузка необходимых ресурсов

# Загрузка необходимых пакетов NLTK
nltk_packages = ['punkt', 'stopwords']
for package in nltk_packages:
    try:
        nltk.data.find(f'tokenizers/{package}')
    except LookupError:
        nltk.download(package)

# Загрузка русских стоп-слов из NLTK
from nltk.corpus import stopwords
russian_stopwords = set(stopwords.words("russian"))

# Загрузка модели spaCy для русского языка
try:
    nlp = spacy.load("ru_core_news_sm")
except OSError:
    print("Модель spaCy 'ru_core_news_sm' не найдена. Устанавливаем...")
    os.system("python -m spacy download ru_core_news_sm")
    nlp = spacy.load("ru_core_news_sm")

# 2. Загрузка данных

# Путь к файлу
file_path = r'C:\Users\vtako\Desktop\Project\NPL homework 1\labeled.csv'

# Загрузка датасета
df = pd.read_csv(file_path)

# Проверка первых строк датасета
print("Первые строки датасета:")
print(df.head())

# 3. Предобработка текста

def preprocess(text):
    if pd.isnull(text):
        return ""
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление пунктуации и цифр
    text = text.translate(str.maketrans('', '', string.punctuation + string.digits))
    return text

# Применение предобработки к колонке "comment"
df['comment_clean'] = df['comment'].apply(preprocess)

# 4. Реализация Стемминга

# Инициализация стеммера для русского языка
stemmer = SnowballStemmer("russian")

def stemming(text):
    if not text:
        return ""
    # Токенизация с использованием spaCy
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
    # Применение стемминга к каждому слову
    stemmed_words = [stemmer.stem(word) for word in tokens]
    return " ".join(stemmed_words)

# Применение стемминга к колонке "comment_clean"
df['comment_stemmed'] = df['comment_clean'].apply(stemming)

# Просмотр результатов стемминга
print("\nПример стемминга:")
print(df[['comment', 'comment_stemmed']].head())

# 5. Реализация Лемматизации

def lemmatization(text):
    if not text:
        return ""
    doc = nlp(text)
    # Собираем леммы, исключая знаки препинания и пробелы
    lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    return " ".join(lemmas)

# Применение лемматизации к колонке "comment_clean"
df['comment_lemmatized'] = df['comment_clean'].apply(lemmatization)

# Просмотр результатов лемматизации
print("\nПример лемматизации:")
print(df[['comment', 'comment_lemmatized']].head())

# 6. Создание Bag of Words (BoW)

# Инициализация CountVectorizer с учётом русских стоп-слов
vectorizer = CountVectorizer(stop_words=russian_stopwords)

# Создание BoW на основе лемматизированных комментариев
X_bow = vectorizer.fit_transform(df['comment_lemmatized'])

# Получение матрицы признаков в виде DataFrame
bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Ограничение количества признаков (опционально)
# Например, оставить только 1000 наиболее частотных слов
# vectorizer = CountVectorizer(stop_words=russian_stopwords, max_features=1000)
# X_bow = vectorizer.fit_transform(df['comment_lemmatized'])
# bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Добавление BoW к исходному датасету
df = pd.concat([df, bow_df], axis=1)

# Просмотр обновленного датасета
print("\nОбновленный датасет с BoW:")
print(df.head())

# 7. Сохранение обработанного датасета (опционально)

processed_file_path = r'C:\Users\vtako\Desktop\Project\NPL homework 1\processed_labeled.csv'
df.to_csv(processed_file_path, index=False)
print(f"\nОбработанный датасет сохранён по пути: {processed_file_path}")

# 8. Дополнительные рекомендации

# Удаление пропущенных значений (если необходимо)
# Например, заполнение пропусков пустыми строками
df['comment'] = df['comment'].fillna('')


                                             comment  \
0               Верблюдов-то за что? Дебилы, бл...\n   
1  Хохлы, это отдушина затюканого россиянина, мол...   
2                          Собаке - собачья смерть\n   
3  Страницу обнови, дебил. Это тоже не оскорблени...   
4  тебя не убедил 6-страничный пдф в том, что Скр...   

                                     stemmed_comment  \
0                      верблюдов то за что дебилы бл   
1  хохлы это отдушина затюканого россиянина мол в...   
2                              собаке собачья смерть   
3  страницу обнови дебил это тоже не оскорбление ...   
4  тебя не убедил 6 страничный пдф в том что скри...   

                                  lemmatized_comment  
0                      верблюдов то за что дебилы бл  
1  хохлы это отдушина затюканого россиянина мол в...  
2                              собаке собачья смерть  
3  страницу обнови дебил это тоже не оскорбление ...  
4  тебя не убедил 6 страничный пдф в том что скри..